# ImageDataGenerator and tf.data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import make_moons
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

assert tf.__version__ == '2.0.0-alpha0'

## Cifar10 Data

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
np.unique(y_train)

## Model

In [ ]:
model = Sequential([
    Reshape((32*32*3,), input_shape=(32, 32, 3)),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(lr=3e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Keras ImageDataGenerator

In [ ]:
idg = ImageDataGenerator(rescale = 1./255)

In [ ]:
batch_size = 256

In [ ]:
datagen = idg.flow(X_train,
                   y=y_train,
                   batch_size=batch_size,
                   shuffle=True)

In [ ]:
images, labels = datagen.next()

In [ ]:
plt.figure()
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])

In [ ]:
model.fit_generator(datagen, epochs=3)

## Tensorflow Dataset

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [ ]:
def rescale(x, y):
    return tf.cast(x, tf.float32)/255., y

In [ ]:
ds = ds.map(rescale)
ds = ds.shuffle(buffer_size=len(X_train)*2)
ds = ds.repeat()
ds = ds.batch(256)

In [ ]:
model.fit(ds,
          steps_per_epoch=len(X_train)/batch_size,
          epochs=3)

## Exercises

The power of TF Dataset is really apparent when we deal with a very large dataset formed of multiple files on disk or when complex pre-processing functions are needed.

Take a look at the documentation [here](https://www.tensorflow.org/alpha/tutorials/load_data/text) and reproduce one of the examples.